In [69]:
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt 
import numpy as np 
import math
import geopandas as gpd
import csv
import statistics

#import geoplot as gplt


from datetime import datetime
from pylab import *
from shapely.ops import nearest_points

In [27]:
df = pd.read_csv('nyc_airbnb.csv')
df2 = pd.read_csv('NYC_Citywide_Annualized_Calendar_Sales_Update.csv')
geo_ny = gpd.read_file('./Individual_Landmark_Lots/Individual_Landmark_Lots.shp')


In [28]:
#CLEAN AIRBNB DATASET (df)

#drop unused column
df.drop(['host_name','host_id','name'], axis=1, inplace=True)

#Only keep data with positive price
df = df[df["price"] > 0]

#drop duplicate
df = df.drop_duplicates()

#check null data
df.isnull().sum()

id                                    0
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10051
reviews_per_month                 10051
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [29]:
#EDIT/ADD COLUMNS TO AIRBNB DATASET (df)

#add activity column
#activity = No Record if no last review
#activity = Inactive if last review is not within 1year of the latest last_review
#activity = Active otherwise

#convert to datetime

df['last_review'] = pd.to_datetime(df['last_review'])

time_threshold = pd.to_datetime('2018/12/06')

df.loc[df['last_review'] >= time_threshold, 'activity'] = 'Active'
df.loc[df['last_review'] < time_threshold, 'activity'] = 'Inactive'
df.loc[df['last_review'].isnull(), 'activity'] = 'No Record'

df_active = df.loc[df['activity'] == 'Active']
df_active

#add occupancy_% column
df_active['occupancy_%'] = round(100 - df_active['availability_365']/ 365 * 100).astype('float')

#reset index
df_active.reset_index(inplace=True)
df_active.drop(['index'], axis=1, inplace=True)



C:\Users\chila\miniconda3\envs\cs_ftmle\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\chila\miniconda3\envs\cs_ftmle\lib\site-packages\pandas\core\frame.py:4164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [30]:
#DETERMINE ON AVERAGE HOW FAR AN AIRBNB FROM ALL THE LANDMARKS OF NYC

def haversine_distance(lat1, lon1, lat2, lon2):
    '''
    function to calculated distance in km based on long and lat
    '''
    r = 6371
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
    res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
    return np.round(res, 2)

def find_distance(df, df2):
    how_far_list = []
    for idx, item in df['coords'].iteritems():
        distance_list = []
        
        for idx2, item2 in df2['coords'].iteritems():
            #print(item, item2)
            l = haversine_distance(item[1], item[0], item2[1], item2[0])
            distance_list.append(l)
            
        mean_dist = mean(distance_list)   
        how_far_list.append(mean_dist)
        if idx % 1000 == 0:
            print(idx)
    return how_far_list

   
# geo_ny = geo_ny[['OBJECTID', 'geometry']]

# geo_ny = geo_ny.to_crs("EPSG:4326")  #convert to correct projection
# geo_ny['coords'] = geo_ny['geometry'].apply(lambda x: x.representative_point().coords[:]) 
# geo_ny['coords'] = [coords[0] for coords in geo_ny['coords']]

# df_geo = df_active[['latitude','longitude']]
# df_geo = gpd.GeoDataFrame(df_geo, geometry=gpd.points_from_xy(df_geo.longitude, df_geo.latitude))

# df_geo['coords'] = df_geo['geometry'].apply(lambda x: x.representative_point().coords[:]) 
# df_geo['coords'] = [coords[0] for coords in df_geo['coords']]


# how_far_km = find_distance(df_geo, geo_ny)

#save to csv so can import csv later to save time
#how_far_df = pd.DataFrame(how_far_km, columns=["how_far_km"])
#how_far_df.to_csv('df_active_how_far_km2.csv', index=False)


In [31]:
#read how_far_km_csv_file and add to df_active
df3 = pd.read_csv('df_active_how_far_km.csv')
df3

df_active = pd.concat([df_active, df3], axis = 1)
df_active.to_csv('clean_nyc_airbnb.csv', index=False)

In [32]:
#CLEAN SALE_PRICE DATASET (df2)

#drop unused column
df2 = df2[['BOROUGH','BUILDING CLASS CATEGORY','NEIGHBORHOOD','SALE PRICE','SALE DATE','Latitude','Longitude']]

#drop duplicate
df2 = df2.drop_duplicates()

#drop null data
df2 = df2.dropna()

#Rename borough

df2.columns = ['neighbourhood_group','building_class','neighbourhood','sale_price','sale_date','latitude','longitude']

df2['neighbourhood_group'] = df2['neighbourhood_group'].astype('str')

df2['neighbourhood_group'].replace('1','Manhattan', inplace = True)
df2['neighbourhood_group'].replace('2','Brooklyn', inplace = True)
df2['neighbourhood_group'].replace('3','Queens', inplace = True)
df2['neighbourhood_group'].replace('4','Bronx', inplace = True)
df2['neighbourhood_group'].replace('5','Staten Island', inplace = True)

In [33]:
#FILTER OUT BUILDING CLASS THAT IS SUITABLE FOR AIRBNB(df2)

airbnb_options = [
    '01 ONE FAMILY DWELLINGS',
    '02 TWO FAMILY DWELLINGS',
    '10 COOPS - ELEVATOR APARTMENTS',
    '13 CONDOS - ELEVATOR APARTMENTS',
    '03 THREE FAMILY DWELLINGS',
    '07 RENTALS - WALKUP APARTMENTS',
    '09 COOPS - WALKUP APARTMENTS',
    '04 TAX CLASS 1 CONDOS',
    '15 CONDOS - 2-10 UNIT RESIDENTIAL',
    '12 CONDOS - WALKUP APARTMENTS',
    '17 CONDO COOPS',
    '14 RENTALS - 4-10 UNIT',
    '08 RENTALS - ELEVATOR APARTMENTS',
    '16 CONDOS - 2-10 UNIT WITH COMMERCIAL UNIT'      
]

df2 = df2[df2['building_class'].isin(airbnb_options)]

In [34]:
#EDIT/ADD COLUMNS TO SALE_PRICE DATASET (df2)

#convert to datetime
df2['sale_date'] = pd.to_datetime(df2['sale_date'])

lower_time_threshold = pd.to_datetime('2018/12/06')
upper_time_threshold = pd.to_datetime('2019/12/06')

df2 = df2.loc[df2['sale_date'] >= lower_time_threshold ]
df_sale = df2.loc[df2['sale_date'] <= upper_time_threshold]

#reset index
df_sale.reset_index(inplace=True)
df_sale.drop(['index'], axis=1, inplace=True)



C:\Users\chila\miniconda3\envs\cs_ftmle\lib\site-packages\pandas\core\frame.py:4164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [41]:
# #DETERMINE ON AVERAGE HOW FAR A SALE PROPERTY FROM ALL THE LANDMARKS OF NYC

# geo_ny = geo_ny[['OBJECTID', 'geometry']]
# geo_ny = geo_ny.to_crs("EPSG:4326")  #convert to correct projection
# geo_ny['coords'] = geo_ny['geometry'].apply(lambda x: x.representative_point().coords[:]) 
# geo_ny['coords'] = [coords[0] for coords in geo_ny['coords']]

# df_geo = df_sale[['latitude','longitude']]

# df_geo = gpd.GeoDataFrame(df_geo, geometry=gpd.points_from_xy(df_geo.longitude, df_geo.latitude))
# df_geo['coords'] = df_geo['geometry'].apply(lambda x: x.representative_point().coords[:]) 
# df_geo['coords'] = [coords[0] for coords in df_geo['coords']]

# how_far_km = find_distance(df_geo, geo_ny)

# print(df_geo['coords'].count)
# print(len(how_far_km))

# #save to csv so can import csv later to save time
# how_far_df = pd.DataFrame(how_far_km, columns=["how_far_km"])
# how_far_df.to_csv('df_sale_how_far_km2.csv', index=False)

In [42]:
#read csv_file and add to df_active
df4 = pd.read_csv('df_sale_how_far_km.csv')
df4

df_sale = pd.concat([df_sale, df4], axis = 1)
df_sale.to_csv('clean_nyc_sale.csv', index = False)

In [58]:
df_active

,id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,activity,occupancy_%,how_far_km,neigh_comp
0,2595,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355,Active,3.0,7.330251,MIDTOWN
1,3831,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-05-07,4.64,1,194,Active,47.0,9.695046,CLINTON HILL
2,5099,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3,74,2019-06-22,0.59,1,129,Active,65.0,7.263750,MURRAY HILL
3,5178,Manhattan,Hell's Kitchen,40.76489,-73.98493,Private room,79,2,430,2019-06-24,3.47,1,220,Active,40.0,7.695761,MIDTOWN WEST
4,5238,Manhattan,Chinatown,40.71344,-73.99037,Entire home/apt,150,1,160,2019-09-06,1.33,4,188,Active,48.0,7.823571,CHINATOWN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25983,36425863,Manhattan,Upper East Side,40.78099,-73.95366,Private room,129,1,1,2019-07-07,1.00,1,147,Active,60.0,8.613975,UPPER EAST SIDE
25984,36427429,Queens,Flushing,40.75104,-73.81459,Private room,45,1,1,2019-07-07,1.00,6,339,Active,7.0,15.835060,FLUSHING
25985,36438336,Staten Island,Great Kills,40.54179,-74.14275,Private room,235,1,1,2019-07-07,1.00,1,87,Active,76.0,26.005483,GREAT KILLS
25986,36442252,Bronx,Mott Haven,40.80787,-73.92400,Entire home/apt,100,1,2,2019-07-07,2.00,1,40,Active,89.0,11.133406,MOTT HAVEN


In [78]:
#DETERMINE THE MEDIAN PRICE OF EACH NEIGHBOURHOOD


#Define a new neighbourhood column to compare with neighbourhood data from sale
df_active['neigh_comp'] = df_active['neighbourhood'].str.upper()


#rename neighbour hood to match sale data
#some neighbourhood are not on the sale list, can assume name of nearby neighbourhood
replace = [
    ['CONCOURSE VILLAGE', 'CONCOURSE'],['WEST BRIGHTON', 'WEST NEW BRIGHTON'],
    ['KEW GARDENS HILLS', 'KEW GARDENS'], ['BAY TERRACE, STATEN ISLAND', 'BAY TERRACE'],
    ['WESTCHESTER SQUARE', 'WESTCHESTER'],["BULL'S HEAD", 'BULLS HEAD'],
    ["NEW DORP BEACH", 'NEW DORP-BEACH'],["BEDFORD-STUYVESANT", 'BEDFORD STUYVESANT'],
    ["EAST HARLEM", 'HARLEM-EAST'],["FLATIRON DISTRICT", 'FLATIRON'],
    ["NORTH RIVERDALE", 'RIVERDALE'],["EAST MORRISANIA", 'MORRISANIA'],
    ["EAST FLATBUSH", 'FLATBUSH-EAST'],["PRINCE'S BAY", 'PRINCES BAY'],
    ["PRINCE'S BAY", 'PRINCES BAY'],["FINANCIAL DISTRICT", 'FINANCIAL'],
    ["SOUTH SLOPE", 'PARK SLOPE SOUTH'],["HELL'S KITCHEN", 'MIDTOWN WEST'],
    ["WEST VILLAGE", 'GREENWICH VILLAGE'],['NOLITA','LITTLE ITALY'],
    ['PROSPECT-LEFFERTS GARDENS','PROSPECT HEIGHTS'],['ROCKAWAY BEACH','ROCKAWAY PARK'],
    ['BAYSWATER','FAR ROCKAWAY'], ['EASTCHESTER', 'WESTCHESTER'],
    ['DITMARS STEINWAY','ASTORIA'],['THEATER DISTRICT','MIDTOWN WEST'],
    ['EDGEMERE','ROCKAWAY PARK'], ['COLUMBIA ST', 'LOWER EAST SIDE'],
    ['BATTERY PARK CITY','FINANCIAL'],['TWO BRIDGES','SOUTHBRIDGE'],
    ['STUYVESANT TOWN','GRAMERCY'],['UNIVERSITY HEIGHTS',"KINGSBRIDGE HTS/UNIV HTS"],
    ['MARBLE HILL', 'KINGSBRIDGE/JEROME PARK'],['NOHO','EAST VILLAGE'],
    ['RANDALL MANOR','WEST NEW BRIGHTON'],['GRANITEVILLE','MARINERS HARBOR'],
    ['HOWLAND HOOK','BLOOMFIELD'],['LIGHTHOUSE HILL','RICHMONDTOWN-LIGHTHS HILL'],
    ['VINEGAR HILL','NAVY YARD'], ['DUMBO','DOWNTOWN-FULTON FERRY'],
    ['SEA GATE', 'CONEY ISLAND'],['DOWNTOWN BROOKLYN','DOWNTOWN-FULTON MALL'],
    ['FORT HAMILTON','DYKER HEIGHTS'],['OLINVILLE', 'WILLIAMSBRIDGE'],
    ['CLAREMONT','MORRISANIA'], ['ALLERTON','PELHAM PARKWAY NORTH'],
    ['SPUYTEN DUYVIL','RIVERDALE'],['EDENWALD','WESTCHESTER'],
    ['WEST FARMS','CROTONA PARK'],['CLASON POINT', 'SOUNDVIEW'],
    ['VILLAGE MORRISANIA VILLAGE','MORRISANIA'],['MORRISANIA VILLAGE', 'MORRISANIA']
]

for i in replace:
    df_active['neigh_comp'] = df_active['neigh_comp'].str.replace(i[0], i[1])

print(df_active['neigh_comp'].nunique())

190


In [79]:
#DETERMINE MEDIAN PRICE OF PROPERTY IN EACH NEIGHBOURHOOD

#Create a dictionary with all the unique neighbourhood as key
#Find all the sale price in the neighbour hood and add to the value

#This code runs TOO LONG so BE CAREFUL 

# neighbour_list = set(df_active['neigh_comp'].to_list())
# print(len(neighbour_list))
# neighbour_dict = {}


# for count, item in enumerate(neighbour_list, start=1):
#     print(count)
#     temp_list = []
#     for index, row in df2.iterrows():
        
#         if item in row['neighbourhood']:
#             price = row['sale_price']
#             temp_list.append(price)
            
#     neighbour_dict[item]= temp_list
    
#     if len(temp_list) == 0:
#         print(item)

# #make a copy just in case
# neighbour_dict2 = neighbour_dict.copy()
# print(len(neighbour_dict))
# print(len(neighbour_dict2))

190
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190


190
190


In [118]:
neighbour_median = {}
neighbour_mean = {}

for key, value in neighbour_dict.items():
    if key not in neighbour_median:
        neighbour_median[key] = statistics.median(value)
    if key not in neighbour_mean:
        neighbour_mean[key] = statistics.mean(value)


In [111]:
df_active2 = df_active.copy()
df_active2

,id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,activity,occupancy_%,how_far_km,neigh_comp
0,2595,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355,Active,3.0,7.330251,MIDTOWN
1,3831,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-05-07,4.64,1,194,Active,47.0,9.695046,CLINTON HILL
2,5099,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3,74,2019-06-22,0.59,1,129,Active,65.0,7.263750,MURRAY HILL
3,5178,Manhattan,Hell's Kitchen,40.76489,-73.98493,Private room,79,2,430,2019-06-24,3.47,1,220,Active,40.0,7.695761,MIDTOWN WEST
4,5238,Manhattan,Chinatown,40.71344,-73.99037,Entire home/apt,150,1,160,2019-09-06,1.33,4,188,Active,48.0,7.823571,CHINATOWN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25983,36425863,Manhattan,Upper East Side,40.78099,-73.95366,Private room,129,1,1,2019-07-07,1.00,1,147,Active,60.0,8.613975,UPPER EAST SIDE
25984,36427429,Queens,Flushing,40.75104,-73.81459,Private room,45,1,1,2019-07-07,1.00,6,339,Active,7.0,15.835060,FLUSHING
25985,36438336,Staten Island,Great Kills,40.54179,-74.14275,Private room,235,1,1,2019-07-07,1.00,1,87,Active,76.0,26.005483,GREAT KILLS
25986,36442252,Bronx,Mott Haven,40.80787,-73.92400,Entire home/apt,100,1,2,2019-07-07,2.00,1,40,Active,89.0,11.133406,MOTT HAVEN


In [112]:
for key, value in neighbour_median.items():
    df_active2.loc[df_active2['neigh_comp'] == key,'sale_median'] = value

for key, value in neighbour_mean.items():
    df_active2.loc[df_active2['neigh_comp'] == key,'sale_mean'] = value

In [113]:
df_active2

,id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,activity,occupancy_%,how_far_km,neigh_comp,sale_median,sale_mean
0,2595,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355,Active,3.0,7.330251,MIDTOWN,733500.0,2.327106e+06
1,3831,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-05-07,4.64,1,194,Active,47.0,9.695046,CLINTON HILL,695000.0,9.334794e+05
2,5099,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3,74,2019-06-22,0.59,1,129,Active,65.0,7.263750,MURRAY HILL,713500.0,1.153138e+06
3,5178,Manhattan,Hell's Kitchen,40.76489,-73.98493,Private room,79,2,430,2019-06-24,3.47,1,220,Active,40.0,7.695761,MIDTOWN WEST,790000.0,3.997647e+06
4,5238,Manhattan,Chinatown,40.71344,-73.99037,Entire home/apt,150,1,160,2019-09-06,1.33,4,188,Active,48.0,7.823571,CHINATOWN,742000.0,1.849008e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25983,36425863,Manhattan,Upper East Side,40.78099,-73.95366,Private room,129,1,1,2019-07-07,1.00,1,147,Active,60.0,8.613975,UPPER EAST SIDE,925000.0,2.229024e+06
25984,36427429,Queens,Flushing,40.75104,-73.81459,Private room,45,1,1,2019-07-07,1.00,6,339,Active,7.0,15.835060,FLUSHING,330000.0,6.560176e+05
25985,36438336,Staten Island,Great Kills,40.54179,-74.14275,Private room,235,1,1,2019-07-07,1.00,1,87,Active,76.0,26.005483,GREAT KILLS,450000.0,3.709442e+05
25986,36442252,Bronx,Mott Haven,40.80787,-73.92400,Entire home/apt,100,1,2,2019-07-07,2.00,1,40,Active,89.0,11.133406,MOTT HAVEN,483325.0,1.218162e+06
